In [9]:
import numpy as np 
import pandas as pd 
import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_scheduler
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [ ]:
#Load datasets
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

# Print the number of unique values in the Category column
print("\nNumber of unique values in Category column:")
print(train_df['Category'].nunique())

# Print the number of unique values in the Misconception column
print("\nNumber of unique values in Misconception column:")
print(train_df['Misconception'].nunique())

In [20]:
print("Number of rows in train_df:", train_df.shape[0])
print("Number of rows in test_df:", test_df.shape[0])

# Print the first few rows of train_df
print("\nFirst few rows of train_df:")
print(train_df.head())

# Print the first few rows of test_df
print("\nFirst few rows of test_df:")
print(test_df.head())

# Print if any rows are null 
print("\nNull values in train_df:")
print(train_df.isnull().sum())

print("\nNull values in test_df:")
print(test_df.isnull().sum())

# Print the number of unique values in the Category column
print("\nNumber of unique values in Category column:")
print(train_df['Category'].nunique())

class_counts = train_df['Category'].value_counts()
class_percentages = train_df['Category'].value_counts(normalize=True) * 100

majority_class_count = class_counts.max()
minority_class_count = class_counts.min()
imbalance_ratio = majority_class_count / minority_class_count
print("Imbalance ratio:", imbalance_ratio)

print("Data type of Category column:", train_df['Category'].dtype)
print("Data type of Misconception column:", train_df['Misconception'].dtype)
print("Data type of QuestionText column:", train_df['QuestionText'].dtype)
print("Data type of StudentExplanation column:", train_df['StudentExplanation'].dtype)

# Print the number of unique values in the Misconception column
print("\nNumber of unique values in Misconception column:")
print(train_df['Misconception'].nunique())

Number of rows in train_df: 36696
Number of rows in test_df: 3

First few rows of train_df:
   row_id  QuestionId                                       QuestionText  \
0       0       31772  What fraction of the shape is not shaded? Give...   
1       1       31772  What fraction of the shape is not shaded? Give...   
2       2       31772  What fraction of the shape is not shaded? Give...   
3       3       31772  What fraction of the shape is not shaded? Give...   
4       4       31772  What fraction of the shape is not shaded? Give...   

           MC_Answer                                 StudentExplanation  \
0  \( \frac{1}{3} \)                  0ne third is equal to tree nineth   
1  \( \frac{1}{3} \)  1 / 3 because 6 over 9 is 2 thirds and 1 third...   
2  \( \frac{1}{3} \)  1 3rd is half of 3 6th, so it is simplee to un...   
3  \( \frac{1}{3} \)        1 goes into everything and 3 goes into nine   
4  \( \frac{1}{3} \)                    1 out of every 3 isn't coloured   



In [21]:
# BEGIN: Load datasets
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')
# END: Load datasets
# BEGIN: Analyze misconceptions
# Preprocessing function
def preprocess_data(df):
    # Fill missing values in StudentExplaination and Misconception with empty strings
    df['StudentExplaination'] = df['StudentExplanation'].fillna("")
    df['Misconception'] = df['Misconception'].fillna("NA")
    
    # Combine QuestionText and StudentExplaination for feature extraction
    df['CombinedText'] = df['QuestionText'] + " " + df['StudentExplaination']
    
    return df

# Preprocess train and test data
train_df = preprocess_data(train_df)
#test_df = preprocess_data(test_df)

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(train_df['CombinedText']).toarray()
#X_test = vectorizer.transform(test_df['CombinedText']).toarray()

# Encode the Category column as the label
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['Category'])
#y_test = label_encoder.transform(test_df['Category'])

# Misconception column for generating misconceptions
misconceptions_train = train_df['Misconception']

#misconceptions_test = test_df['Misconception']

# Split train data into training and validation sets
X_train, X_val, y_train, y_val, misconceptions_train, misconceptions_val = train_test_split(
    X_train, y_train, misconceptions_train, test_size=0.2, random_state=42
)

# Output the shapes of the datasets
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
#print("Test data shape:", X_test.shape)
print("Labels shape:", y_train.shape)

Training data shape: (29356, 5000)
Validation data shape: (7340, 5000)
Labels shape: (29356,)


In [ ]:
# Shrijan is working here 